In [3]:
# オリジナルモジュールのインポート
from lib.introngap import PileUp
from lib.gbkparse import Seq_count

# モジュールのインポート
import itertools
import logomaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
gbk.read_gbk('../data/gbk/human_ttn.gb')

デフォルト値として、最もエクソンの多いNM_001267550.2を設定


In [16]:
n = 5
# すべてのイントロンの3-prime側のn塩基の配列を取得
right_edges = {}
for i in range(gbk.intron_num()):
    right_edges[i+1] = str(gbk.intron_seq(i+1)[-n:])
# すべてのイントロンの5-prime側のn塩基の配列を取得
left_edges = {}
for i in range(gbk.intron_num()):
    left_edges[i+1] = str(gbk.intron_seq(i+1)[:n])

In [ ]:
# left_edgesとright_edgesの組み合わせを作成
# left_edgesの番号は常にright_edgesの番号より大きいこととする
key_combinations = []
for i in right_edges.keys():
    for j in left_edges.keys():
        if j > i:
            key_combinations.append((i, j))

seq_combinations = []
for i,j in key_combinations:
    seq_combinations.append(right_edges[i] + left_edges[j])

[(1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 24),
 (1, 25),
 (1, 26),
 (1, 27),
 (1, 28),
 (1, 29),
 (1, 30),
 (1, 31),
 (1, 32),
 (1, 33),
 (1, 34),
 (1, 35),
 (1, 36),
 (1, 37),
 (1, 38),
 (1, 39),
 (1, 40),
 (1, 41),
 (1, 42),
 (1, 43),
 (1, 44),
 (1, 45),
 (1, 46),
 (1, 47),
 (1, 48),
 (1, 49),
 (1, 50),
 (1, 51),
 (1, 52),
 (1, 53),
 (1, 54),
 (1, 55),
 (1, 56),
 (1, 57),
 (1, 58),
 (1, 59),
 (1, 60),
 (1, 61),
 (1, 62),
 (1, 63),
 (1, 64),
 (1, 65),
 (1, 66),
 (1, 67),
 (1, 68),
 (1, 69),
 (1, 70),
 (1, 71),
 (1, 72),
 (1, 73),
 (1, 74),
 (1, 75),
 (1, 76),
 (1, 77),
 (1, 78),
 (1, 79),
 (1, 80),
 (1, 81),
 (1, 82),
 (1, 83),
 (1, 84),
 (1, 85),
 (1, 86),
 (1, 87),
 (1, 88),
 (1, 89),
 (1, 90),
 (1, 91),
 (1, 92),
 (1, 93),
 (1, 94),
 (1, 95),
 (1, 96),
 (1, 97),
 (1, 98),
 (1, 99),
 (1, 100),
 (1, 101),
 (1, 1

In [ ]:
key_combinations = list(itertools.product(right_edges.keys(), left_edges.keys()))
seq_combinations = list(itertools.product(right_edges.values(), left_edges.values()))

intron_numb_combinations = [f"{i}_{j}" for i, j in key_combinations]
intron_seq_combinations = [f"{i}_{j}" for i, j in seq_combinations]

In [ ]:
# ワンホットエンコーディングを行う関数
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[s] for s in seq]).flatten()

# エンコーディングされた配列を準備
encoded_sequences = np.array([one_hot_encode(seq) for seq in intron_seq_combinations])

# 主成分分析（PCA）
pca = PCA(n_components=2)  # 2つの主成分を取得
principal_components = pca.fit_transform(encoded_sequences)

# 主成分をデータフレームに変換
df_pca = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# 結果をプロット
fig = px.scatter(df_pca, x='PC1', y='PC2', hover_name=intron_seq_combinations)
fig.show()